In [50]:
import re
import json
import pandas as pd
import glob
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from datetime import datetime
import textstat
from Class_sentiments import Text_Sentiment

In [51]:
#data = import_articles(r'C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_1p8.txt')
#data = import_articles(r'C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_1p0.txt') # Bild
#liste mit Dateinamen und dann schleife (for)
def import_articles(datei):
    with open(datei, encoding='utf-8') as json_file:
        data = json.load(json_file)
    return data

def is_empty(string):
    if string==None:
        return True
    return False

def count_words(string):
    try:
        count = len(string.split())
    except:
        return False        
    return count

def anzahl_kommentare(thread):
    if thread==None:
        return 0
    return len(thread)

def clean_string(string):
    try: 
        stringclean = re.sub(r'[@_!#$;“„"%^§&-*"()<>?/\|}{~:]',' ',string)
    except:
        return False # wenn return in def dann def beendet!
    return(stringclean)

def convert_datum(datum_str):
    try:
        #datum = data['date'].split()[1] # Spiegel
        datum = data['date'] # Bild
        
    except:
        return False
    return pd.to_datetime(datum)

def lesbarkeit(string):
    lesbar = textstat.flesch_reading_ease(string)
    return lesbar

    

In [52]:
liste_id = [] # zur Identifikation der Dateinamen
liste_counts=[] # Anzahl der Textbausteine
liste_kommentare=[] # Anzahl der Kommentare
liste_monate=[] # Anzahl Monate
liste_Jahre=[] # Anzahl Jahre
liste_day=[]

#liste = glob.glob(r'C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\*.txt') # liste Spiegel
liste = glob.glob(r'C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\*.txt') # liste Bild
liste = sorted(liste)
for path in liste:
    #import Datafiles
    data = import_articles(path)
    if is_empty(data['text']):
        continue
    liste_id.append(path.split('\\')[-1])
    #Anzahl Textbausteine
    liste_counts.append(count_words(data['text']))
    #Anzahl Kommentare
    liste_kommentare.append(anzahl_kommentare(data['thread']))
    liste_monate.append(convert_datum(data['date']).month)
    liste_Jahre.append(convert_datum(data['date']).year)
    liste_day.append(convert_datum(data['date']).day)
    
    

In [53]:
liste_lesbarkeit = []
liste_polarity = []
liste_subject=[]

for x in liste:
    string = import_articles(x)
    print(x)
    if is_empty(string['text']):
        continue
    string = clean_string(string['text'])
    ts = Text_Sentiment(string)
    polarity_ts = ts.text_sentiment()[0] # Textsentimentsanalyse Polarity
    subject_ts = ts.text_sentiment()[1] # Textsentimentsanalyse Sentiments
    liste_polarity.append(polarity_ts)
    liste_subject.append(subject_ts)    
    lesbarkeit(string)
    liste_lesbarkeit.append(lesbarkeit(string))

C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p0.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p1.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p2.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p3.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p4.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p5.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p6.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p7.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p8.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\bild_corpus_10p\b_article_10p9.txt
C:\Users\Stefan Kostelecky\Doc

In [54]:
#Erzeuge DataFrame
df = pd.DataFrame({'Id':pd.Series(liste_id),
                   'word_counts':pd.Series(liste_counts),
                   'thread_counts':pd.Series(liste_kommentare),
                   'lesbarkeit':pd.Series(liste_lesbarkeit),
                   'polarity':pd.Series(liste_polarity),
                   'subjectivity': pd.Series(liste_subject),
                   'articles_counts':1,
                   'day':pd.Series(liste_day),
                   'month':pd.Series(liste_monate),
                   'year':pd.Series(liste_Jahre)})

#index
#df.set_index(['Id','month','year','day'], inplace=True)
df.to_csv('df_gesamt_bild.csv')
#man-withney-test

In [6]:
liste_comment_pol = [] # die Liste enthält eine nx4 Matrix (Werte: Min, Max, Median,Mean)
liste_comment_subj = []
liste_Id_sentiments=[]
for path in liste:
    data = import_articles(path)
    print(path)
    liste_Id_sentiments.append(path.split('\\')[-1])
    if is_empty(data['text']):
        continue
    if data['thread']==np.nan or data['thread']==None or len(data['thread'])==0:
        liste_comment_pol.append([np.nan, np.nan,np.nan,np.nan])
        liste_comment_subj.append([np.nan, np.nan,np.nan,np.nan])

    else:
        comment_sentiments=[]
        for comment in data['thread']:
            tc = Text_Sentiment(comment[0])
            comment_sentiments.append(tc.text_sentiment()) # hier entsteht eine liste mit Tuple Einträgen (polarity and subje)
        comment_sentiments = np.array(comment_sentiments) # deshalb wird umgewandelt in Array (matrix n x 2)
        mean = np.mean(comment_sentiments,axis=0) # axis=0, die Spalten werden aggregiert
        minimum = np.min(comment_sentiments,axis=0)
        maximum = np.max(comment_sentiments,axis=0)
        median = np.median(comment_sentiments,axis=0)
        liste_comment_pol.append([mean[0],minimum[0],maximum[0],median[0]])
        liste_comment_subj.append([mean[1],minimum[1],maximum[1],median[1]])        

C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p0.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p1.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p2.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p3.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p4.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p5.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p6.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p7.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p8.txt
C:\Users\Stefan Kostelecky\Documents\FHWN\DQDA\Projekt Python\speigel_corpus_99p\article_10p9.txt
C:\Users\Stefan Kost

In [7]:
#Erstellung eines DataFrame_Polarity_threads
id_var_pol = ['mean_comment_pol','min_comment_pol','max_comment_pol','median_comment_pol'] # Labels für DataFrame
df_pol = pd.DataFrame({'Id':pd.Series(liste_Id_sentiments), # generiert aus Liste ID Sentiments ein pd.Series
                   'comment_pol':pd.Series(liste_comment_pol)}) # generiert aus Liste Comments Polarity ein pd.Series

df_com_pol = df_pol.comment_pol.apply(pd.Series) # Spalte Comment_pol enthält eine Liste als Eintrag -> Liste wird in ein Series umgewandelt
df_com_pol.set_axis(labels=id_var_pol, axis=1) # Beschriftung Labels
df_com_pol = df_com_pol.merge(df_pol,left_index=True, right_index=True)\
.drop(['comment_pol'],axis=1)

#Erstellung eines DataFrame_Subjectivity_threads
id_var_sub = ['mean_comment_sub','min_comment_sub','max_comment_sub','median_comment_sub'] # Labels für DataFrame
df_sub = pd.DataFrame({'Id':pd.Series(liste_Id_sentiments), # generiert aus Liste ID Sentiments ein pd.Series
                   'comment_sub':pd.Series(liste_comment_subj)}) # generiert aus Liste Comments Polarity ein pd.Series

df_com_sub = df_sub.comment_sub.apply(pd.Series) # mit Apply Liste wird in Spalten kopiert
df_com_sub.set_axis(labels=id_var_sub, axis=1) # Beschriftung Labels
df_com_sub = df_com_sub.merge(df_sub,left_index=True,right_index=True)\
.drop(['comment_sub'],axis=1)

#Set Index
df_com_sub.set_index('Id',inplace=True)
df_com_pol.set_index('Id',inplace=True)

#join Tables
df_sent = df_com_sub.join(df_com_pol)
df_sent.to_csv('Sentiments.csv')


In [ ]:
#np_array_pol = np.array(liste_comment_pol) # array
#np_array_pol[:,0]

In [ ]:
#df.word_counts.quantile(0.25)